<a href="https://colab.research.google.com/github/ImperatorAvrelianvs/Generalized_Pell_equation_solver/blob/main/Pell_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from typing import List, Tuple

def continued_fraction_pqa(D: int, P0: int, Q0: int):
    """
    Generator for the PQa algorithm with (P, Q) starting values.
    Yields tuples (i, G_prev, B_prev) exactly at the first index i>=1
    where Q_i = ±1, returning the *previous* (G,B).
    """
    P, Q = P0, Q0
    # Histories for convergents A, B, G:
    A_prev2, A_prev1 = 0, 1
    B_prev2, B_prev1 = 1, 0
    G_prev2, G_prev1 = -P0, Q0

    i = 0
    while True:
        a = int(math.floor((P + math.sqrt(D)) / Q))
        A = a * A_prev1 + A_prev2
        B = a * B_prev1 + B_prev2
        G = a * G_prev1 + G_prev2

        if i >= 1 and abs(Q) == 1:
            # Return index i and the *previous* G,B (i-1)
            yield i, G_prev1, B_prev1
            return

        # Advance P, Q
        P_next = a * Q - P
        Q_next = (D - P_next * P_next) // Q

        # Shift histories
        A_prev2, A_prev1 = A_prev1, A
        B_prev2, B_prev1 = B_prev1, B
        G_prev2, G_prev1 = G_prev1, G
        P, Q = P_next, Q_next
        i += 1

def solve_pell_minus1(D: int) -> Tuple[int,int]:
    """
    Find minimal positive (t,u) solving t^2 - D u^2 = -1, if it exists.
    """
    a0 = int(math.isqrt(D))
    m, d, a = 0, 1, a0
    h2, h1 = 1, a
    k2, k1 = 0, 1

    while True:
        if h1*h1 - D*k1*k1 == -1:
            return h1, k1
        m = d*a - m
        d = (D - m*m) // d
        a = (a0 + m) // d
        h2, h1 = h1, a*h1 + h2
        k2, k1 = k1, a*k1 + k2

def generalized_pell_LMM(D: int, N: int) -> List[Tuple[int,int]]:
    """
    Solve x^2 - D y^2 = N using the LMM algorithm.
    Returns one primitive solution per equivalence class.
    """
    sols: List[Tuple[int,int]] = []
    absN = abs(N)

    # Loop over all f with f^2 | N
    for f in range(1, int(math.isqrt(absN)) + 1):
        if N % (f*f) != 0:
            continue
        m = N // (f*f)
        mod = abs(m)

        # Find all z with z^2 ≡ D mod |m| and -|m|/2 < z ≤ |m|/2
        for z in range(-mod//2 + 1, mod//2 + 1):
            if (z*z - D) % mod != 0:
                continue

            # Run PQa until Q_i = ±1
            for i, r, s in continued_fraction_pqa(D, z, mod):
                val = r*r - D*s*s
                if val == m:
                    sols.append((f*r, f*s))
                elif val == -m:
                    tup = solve_pell_minus1(D)
                    if tup:
                        t, u = tup
                        x = f*(r*t + s*u*D)
                        y = f*(r*u + s*t)
                        sols.append((x, y))
                break  # proceed to next z

    # Deduplicate
    return list(set(sols))

def solve_pell_unit(D: int) -> Tuple[int,int]:
    """
    Find the minimal positive solution (u,v) to u^2 - D v^2 = 1.
    """
    a0 = int(math.isqrt(D))
    m, d, a = 0, 1, a0
    h2, h1 = 1, a
    k2, k1 = 0, 1

    while True:
        if h1*h1 - D*k1*k1 == 1:
            return h1, k1
        m = d*a - m
        d = (D - m*m) // d
        a = (a0 + m) // d
        h2, h1 = h1, a*h1 + h2
        k2, k1 = k1, a*k1 + k2

def main():
    D = int(input("Enter non‐square D > 1: "))
    if int(math.isqrt(D))**2 == D:
        print("Error: D must not be a perfect square.")
        return

    N = int(input("Enter nonzero N: "))
    solutions = generalized_pell_LMM(D, N)
    if not solutions:
        print(f"No integer solutions to x^2 - {D}y^2 = {N}.")
        return

    print(f"\nFound {len(solutions)} primitive solution(s) (one per equivalence class):")
    for x0, y0 in sorted(solutions, key=lambda t: (abs(t[0]), abs(t[1]))):
        print(f"  x = {x0},  y = {y0}")

    count = int(input("\nHow many solutions per equivalence class would you like? "))

    # Find fundamental unit (u,v) for x^2 - D y^2 = 1
    u, v = solve_pell_unit(D)
    print(f"\nUsing fundamental unit (u, v) = ({u}, {v}) to generate solutions...\n")

    all_sols: List[Tuple[int,int]] = []
    for (x0, y0) in solutions:
        # (u_k, v_k) = (1,0) initially
        uk, vk = 1, 0
        for k in range(count):
            xk = x0*uk + y0*vk*D
            yk = x0*vk + y0*uk
            all_sols.append((xk, yk))
            # multiply (u_k, v_k) by (u, v)
            uk, vk = uk*u + vk*v*D, uk*v + vk*u

    # Deduplicate and sort globally
    all_sols = sorted(set(all_sols), key=lambda t: (abs(t[0]), abs(t[1])))

    print(f"Extended solutions to x^2 - {D}y^2 = {N}:\n")
    for x, y in all_sols:
        print(f"  x = {x},  y = {y}")

if __name__ == "__main__":
    main()


Enter non‐square D > 1: 12
Enter nonzero N: 18
